In [36]:
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
from IPython.display import FileLink, display
import qrcode

def _fit_font(c, text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    size = max_font
    while size > min_font and pdfmetrics.stringWidth(t, font_name, size) > box_w:
        size -= 0.5
    return max(min_font, size)

def _qr_reader(value, box_size=6, border=2):
    q = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=box_size, border=border)
    q.add_data(value); q.make(fit=True)
    return ImageReader(q.make_image(fill_color="black", back_color="white").convert("RGB"))

def _neat(v):
    if v in (None, "", "None"): return ""
    try:
        f = float(str(v)); s = f"{f:.2f}".rstrip("0").rstrip("."); return s
    except: return str(v)

def render_addstock_form(cfg):
    THEME = {
        "page_size": cfg.get("page_size", A4),
        "margins_mm": cfg.get("margins_mm", 14),
        "pad_mm": cfg.get("pad_mm", 3),
        "gap_mm": cfg.get("gap_mm", 6),
        "corner": cfg.get("corner", 4),
        "stroke": cfg.get("stroke", 1.0),
        "label_font": cfg.get("label_font", "Helvetica"),
        "value_font": cfg.get("value_font", "Helvetica-Bold"),
        "title_font": cfg.get("title_font", "Helvetica-Bold"),
        "label_color": cfg.get("label_color", colors.grey),
        "value_color": cfg.get("value_color", colors.black),
        "label_sz": cfg.get("label_sz", 9.2),
        "value_max": cfg.get("value_max", 12.8),
        "value_min": cfg.get("value_min", 8.8),
        "title_max": cfg.get("title_max", 22),
        "title_min": cfg.get("title_min", 14),
        "label_top_pad_mm": cfg.get("label_top_pad_mm", 2.6),
        "label_val_gap_mm": cfg.get("label_val_gap_mm", 3.6),
        "value_left_pad_mm": cfg.get("value_left_pad_mm", 2.0),
        "row_vpad_mm": cfg.get("row_vpad_mm", 2.0),
    }

    LAYOUT = {
        "title": cfg.get("title", "ADDSTOCK FORM"),
        "filename": cfg.get("filename", "addstock_form_scalable.pdf"),
        "qr_key": cfg.get("qr_key", "current_form_qr_code"),
        "rows": [
            {
                "row_h_mm": cfg.get("row_h_mm", 14),
                "fields": [
                    {"key":"id","label":"Form ID","ratio":0.3},
                    {"key":"current_form_qr_code","label":"Current QR","ratio":0.30},
                    {"key":"previous_form_qr_code","label":"Previous QR","ratio":0.30},

                ],
            },
            {
                "row_h_mm": cfg.get("row_h_mm", 14),
                "fields": [
                    {"key":"supplier","label":"Supplier","ratio":.4},
                    {"key":"delivery_receipt_number","label":"DR #","ratio":0.2},
                    {"key":"date","label":"Date (MongoDB)","ratio":0.2},
                ],
            },
        ],
        "items": {
            "breakpoints": [
                {"min_count":0, "cols":3, "gap_col_mm":6},
                {"min_count":10,"cols":2, "gap_col_mm":8},
                {"min_count":18,"cols":1, "gap_col_mm":10},
            ],
            "title_h_mm": cfg.get("item_title_h_mm", 9),
            "row_h_mm": cfg.get("item_row_h_mm", 8),
            "rows": [
                {"key":"product", "label":"Product", "fmt":str},
                {"key":"quantity", "label":"Quantity (L)", "fmt":_neat},
                {"key":"gauge_before_addstock","label":"Gauge Before","fmt":_neat},
                {"key":"gauge_after_addstock","label":"Gauge After","fmt":_neat},
            ],
            "lab_ratio": cfg.get("item_lab_ratio", 0.46),
        },
        "footer_rows": [
            {
                "row_h_mm": cfg.get("footer_row_h_mm", 14),
                "fields": [
                    {"key":"cashier_employee_number","label":"Cashier Emp. No.","ratio":0.50},
                    {"key":"recorder_employee_number","label":"Recorder Emp. No.","ratio":0.50},
                ],
            }
        ]
    }

    prefill = dict(cfg.get("prefill", {}))
    items = list(prefill.get("items", []))

    w, h = THEME["page_size"]
    m = THEME["margins_mm"]*mm
    pad = THEME["pad_mm"]*mm
    gap = THEME["gap_mm"]*mm
    left, right = m, w - m
    usable_w = right - left

    c = canvas.Canvas(LAYOUT["filename"], pagesize=THEME["page_size"])
    c.setTitle(LAYOUT["title"])

    def title_block(cursor_y):
        fs = _fit_font(c, LAYOUT["title"], THEME["title_font"], THEME["title_max"], THEME["title_min"], w - 2*m)
        c.setFont(THEME["title_font"], fs)
        top_y = h - m - 2*mm
        c.drawCentredString(w/2, top_y, LAYOUT["title"])
        code = str(prefill.get(LAYOUT["qr_key"]) or "")
        y = top_y - fs - 2*mm
        if code:
            qr = _qr_reader(code, box_size=5, border=2)
            box = 18*mm
            x = (w - box) / 2
            c.drawImage(qr, x, y - box, width=box, height=box, preserveAspectRatio=True, anchor='sw', mask='auto')
            c.setFont(THEME["value_font"], 10.5)
            c.drawCentredString(w/2, y - box - 9, code)
            return y - box - 16
        return y - 10

    def draw_row(cursor_y, row_spec):
        row_h = row_spec["row_h_mm"]*mm
        c.setLineWidth(THEME["stroke"])
        c.roundRect(left, cursor_y - row_h, usable_w, row_h, THEME["corner"], stroke=1, fill=0)
        acc = 0.0
        x = left
        inner_top = cursor_y - THEME["row_vpad_mm"]*mm
        inner_bottom = cursor_y - row_h + THEME["row_vpad_mm"]*mm
        for i, f in enumerate(row_spec["fields"]):
            acc += f["ratio"]
            if i < len(row_spec["fields"]) - 1:
                x_next = left + usable_w * acc
                c.line(x_next, cursor_y - row_h, x_next, cursor_y)
            col_w = usable_w * f["ratio"]
            c.setFillColor(THEME["label_color"])
            lfs = _fit_font(c, f["label"], THEME["label_font"], THEME["label_sz"], 8.5, col_w - 2*pad)
            c.setFont(THEME["label_font"], lfs)
            ly = inner_top - THEME["label_top_pad_mm"]*mm - lfs*0.05
            c.drawString(x + pad, ly, f["label"])
            val = prefill.get(f["key"])
            val = "" if val is None else str(val)
            if val:
                c.setFillColor(THEME["value_color"])
                vfs = _fit_font(c, val, THEME["value_font"], THEME["value_max"], THEME["value_min"], col_w - 2*pad - THEME["value_left_pad_mm"]*mm)
                vy = ly - THEME["label_val_gap_mm"]*mm - vfs*0.05
                if vy < inner_bottom + 2: vy = inner_bottom + 2
                c.setFont(THEME["value_font"], vfs)
                c.drawString(x + pad + THEME["value_left_pad_mm"]*mm, vy, val)
            x = left + usable_w * acc
        return cursor_y - row_h

    def paginate(cursor_y, need_mm):
        need = need_mm*mm
        if cursor_y - need < m + 28*mm:
            c.showPage(); c.setTitle(LAYOUT["title"])
            return title_block(cursor_y)
        return cursor_y

    def choose_cols(n):
        bps = sorted(LAYOUT["items"]["breakpoints"], key=lambda z: z["min_count"], reverse=True)
        for bp in bps:
            if n >= bp["min_count"]: return bp["cols"], bp["gap_col_mm"]*mm
        return 3, 6*mm

    def draw_items(cursor_y, rows):
        n = len(rows)
        cols, gap_col = choose_cols(n)
        card_w = (usable_w - gap_col*(cols-1)) / cols
        title_h = LAYOUT["items"]["title_h_mm"]*mm
        row_h = LAYOUT["items"]["row_h_mm"]*mm
        block_h = title_h + gap + len(LAYOUT["items"]["rows"])*row_h + gap
        y_top = cursor_y
        for start in range(0, n, cols):
            if y_top - block_h < m + 40*mm:
                c.showPage(); c.setTitle(LAYOUT["title"])
                y_top = title_block(y_top) - gap
            x = left
            batch = rows[start:start+cols]
            for idx, it in enumerate(batch):
                c.setLineWidth(THEME["stroke"])
                c.roundRect(x, y_top - block_h, card_w, block_h, THEME["corner"], stroke=1, fill=0)
                c.setFont(THEME["value_font"], 10.4)
                c.drawString(x + pad, y_top - title_h + 2, f"Item {start+idx+1}")
                y = y_top - title_h - gap
                lab_w = card_w * LAYOUT["items"]["lab_ratio"]
                val_w = card_w - lab_w
                c.setStrokeColor(colors.lightgrey); c.setLineWidth(0.6)
                c.line(x + lab_w, y_top - block_h + 2.5*mm, x + lab_w, y_top - 2.5*mm)
                c.setStrokeColor(colors.black); c.setLineWidth(THEME["stroke"])
                for r_i, r in enumerate(LAYOUT["items"]["rows"]):
                    row_top = y
                    row_mid = row_top - (row_h/2)
                    c.setFillColor(THEME["label_color"])
                    c.setFont(THEME["label_font"], 9.2)
                    c.drawString(x + pad, row_mid + 3, r["label"])
                    c.setFillColor(THEME["value_color"])
                    raw_val = it.get(r["key"], "")
                    sval = r["fmt"](raw_val) if callable(r["fmt"]) else str(raw_val)
                    fs = _fit_font(c, str(sval), THEME["value_font"], 12.2, 8.6, val_w - 2*pad)
                    tw = pdfmetrics.stringWidth(str(sval), THEME["value_font"], fs)
                    c.setFont(THEME["value_font"], fs)
                    c.drawString(x + lab_w + pad + max(0, (val_w - 2*pad - tw)), row_mid + 3, str(sval))
                    if r_i < len(LAYOUT["items"]["rows"]) - 1:
                        c.setStrokeColor(colors.lightgrey); c.setLineWidth(0.6)
                        c.line(x + 1.5*mm, row_top - row_h + 1, x + card_w - 1.5*mm, row_top - row_h + 1)
                        c.setStrokeColor(colors.black); c.setLineWidth(THEME["stroke"])
                    y -= row_h
                x += card_w + gap_col
            y_top -= block_h
        if n == 0:
            c.setDash(3, 3); c.rect(left, y_top - 18*mm, usable_w, 18*mm, stroke=1, fill=0); c.setDash()
            c.setFont("Helvetica-Oblique", 9.2)
            c.drawCentredString(w/2, y_top - 10*mm, "No items")
            y_top -= 18*mm
        return y_top

    def draw_total(cursor_y, rows):
        try: s = sum([float(str(i.get("quantity"))) for i in (rows or []) if str(i.get("quantity")) not in ("","None")])
        except: s = ""
        row_h = 13*mm
        lab_w = usable_w*0.70
        if cursor_y - row_h < m + 24*mm:
            c.showPage(); c.setTitle(LAYOUT["title"])
            cursor_y = title_block(cursor_y) - gap
        c.setLineWidth(THEME["stroke"])
        c.roundRect(left, cursor_y - row_h, usable_w, row_h, THEME["corner"], stroke=1, fill=0)
        c.line(left + lab_w, cursor_y - row_h, left + lab_w, cursor_y)
        c.setFont(THEME["value_font"], 11.7)
        c.drawString(left + pad, cursor_y - row_h/2 + 4, "Total Quantity (L)")
        t = _neat(s); fs = 13.2
        tw = pdfmetrics.stringWidth(t, THEME["value_font"], fs)
        c.setFont(THEME["value_font"], fs)
        c.drawString(left + lab_w + pad + (usable_w - lab_w - 2*pad - tw), cursor_y - row_h/2 + 4, t)
        return cursor_y - row_h

    cursor = title_block(h - m)
    for row in LAYOUT["rows"]:
        need = row["row_h_mm"] + THEME["gap_mm"]
        cursor = paginate(cursor, need)
        cursor = draw_row(cursor - THEME["gap_mm"]*mm, row) - THEME["gap_mm"]*mm

    cursor = draw_items(cursor - 2, items) - 8
    cursor = draw_total(cursor, items) - 8

    for row in LAYOUT["footer_rows"]:
        need = row["row_h_mm"] + THEME["gap_mm"]
        cursor = paginate(cursor, need)
        cursor = draw_row(cursor - THEME["gap_mm"]*mm, row) - THEME["gap_mm"]*mm

    c.setFont("Helvetica", 8.6); c.setFillColor(colors.grey)
    c.drawCentredString(w/2, m - 3*mm, "Page 1")
    c.setFillColor(colors.black)
    c.save()
    try: display(FileLink(LAYOUT["filename"]))
    except: pass
    return os.path.abspath(LAYOUT["filename"])

example = {
    "id":"AS-2025-0005",
    "current_form_qr_code":"ASQR-0005",
    "previous_form_qr_code":"ASQR-0004",
    "delivery_receipt_number":"DR-99855",
    "supplier":"Shell Philippines",
    "date":"2025-10-09T00:00:00Z",
    "items":[
        {"product":"Diesel","quantity":2500,"gauge_before_addstock":10.2,"gauge_after_addstock":13.4},
        {"product":"Regular","quantity":1700,"gauge_before_addstock":8.9,"gauge_after_addstock":10.8},
        {"product":"Premium","quantity":950,"gauge_before_addstock":6.2,"gauge_after_addstock":7.5}
    ],
    "cashier_employee_number":"99887",
    "recorder_employee_number":"12345"
}

out_path = render_addstock_form({
    "title":"ADDSTOCK FORM",
    "filename":"addstock_form_footer_empnos.pdf",
    "prefill": example,
    "row_h_mm": 14,
    "item_row_h_mm": 8.5,
    "label_top_pad_mm": 3.0,
    "label_val_gap_mm": 4.0
})
print(out_path)


h:\github4\jefstore-gasstations-backend\reportlab\addstocks\filled\addstock_form_footer_empnos.pdf

h:\github4\jefstore-gasstations-backend\reportlab\addstocks\filled\addstock_form_footer_empnos.pdf
